In [64]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets, interact
from scipy.signal import freqz
%matplotlib inline

In [65]:
audio_data = pd.read_csv("audio_features.csv")
metadata_df = pd.read_csv("metadata.csv")
all_data = pd.read_csv("all_features.csv")

In [ ]:
display(all_data.head())
display(metadata_df.head())

,smcAC_case001,smcDC_case001,vib_table_case001,vib_spindle_case001,AE_table_case001,AE_spindle_case001,smcAC_case002,smcDC_case002,vib_table_case002,vib_spindle_case002,...,AE_table_case164,AE_spindle_case164,smcAC_case165,smcDC_case165,vib_table_case165,vib_spindle_case165,AE_table_case165,AE_spindle_case165,smcAC_case166,smcDC_case166
0,-0.017090,0.625000,0.078125,0.314941,0.087280,0.103760,0.307617,0.668945,0.075684,0.301514,...,0.114746,0.139771,0.244141,1.328125,0.063477,0.290527,0.101318,0.117798,-0.205078,1.381836
1,0.263672,0.810547,0.085449,0.301514,0.098267,0.123291,0.312500,0.678711,0.080566,0.308838,...,0.095215,0.112305,0.244141,1.333008,0.058594,0.279541,0.121460,0.140381,-0.239258,1.386719
2,0.207520,0.781250,0.078125,0.303955,0.092163,0.104980,0.195312,0.859375,0.078125,0.299072,...,0.101929,0.120239,0.205078,1.333008,0.058594,0.285645,0.122070,0.142822,-0.156250,1.386719
3,0.302734,0.849609,0.073242,0.300293,0.095215,0.111084,0.187988,1.123047,0.080566,0.306396,...,0.098877,0.118408,0.156250,1.333008,0.063477,0.286865,0.119019,0.139160,0.126953,1.381836
4,0.239258,1.098633,0.083008,0.299072,0.083008,0.092163,-0.009766,1.166992,0.075684,0.319824,...,0.098877,0.120239,0.048828,1.337891,0.058594,0.286865,0.114136,0.132446,0.200195,1.376953


,Unnamed: 0,case,run,VB,time,DOC,feed,material
0,NaN,1.0,1.0,0.0,2.0,1.5,0.5,1.0
1,NaN,1.0,2.0,NaN,4.0,1.5,0.5,1.0
2,NaN,1.0,3.0,NaN,6.0,1.5,0.5,1.0
3,NaN,1.0,4.0,11.0,7.0,1.5,0.5,1.0
4,NaN,1.0,5.0,NaN,11.0,1.5,0.5,1.0


In [69]:
signal = audio_data["AE_spindle_case001"] # Choose a signal to analyze

In [70]:
def sTFT(time_seg, filter_index, case_number, signal_location, category=None):
    signal_location_dict = {0: "spindle", 1: "table"}
    loc_str = signal_location_dict[signal_location]
    case_str = str(case_number).zfill(3)
    signal = audio_data[f"AE_{loc_str}_case{case_str}"]
    
    Nwindow = N // time_seg
    time_freq_image = np.zeros((N // 2, time_seg))

    for time_index in range(time_seg):
        signal_pad = np.zeros(N)
        window = filters[filter_index]["function"](Nwindow) if filter_index < 2 else filters[filter_index]["function"](Nwindow, 10)
        segment = signal[time_index * Nwindow:(time_index + 1) * Nwindow] * window
        signal_pad[:Nwindow] = segment
        signal_freq = np.fft.fft(signal_pad)[:N // 2]
        signal_freq_max = np.max(np.abs(signal_freq))
        signal_freq = np.abs(signal_freq) / signal_freq_max
        time_freq_image[:, time_index] = 20 * np.log10(signal_freq)

    w, h = plt.figaspect(0.4)
    plt.figure(figsize=(w, h))
    plt.imshow(time_freq_image[::-1, :], cmap="gray", extent=(0, N / fs * 1000, 0, fs // 2), aspect='auto', clim=(-60, 0))
    
    title = f"Time-Frequency Analysis — Case {case_str} ({loc_str.capitalize()})"
    if category:
        title += f" — {category}"
    
    plt.title(title)
    plt.xlabel("Time (ms)")
    plt.ylabel("Frequency (Hz)")
    plt.show()


In [74]:
# label_dict

# Analyze STFT of Each Label

In [75]:
# bins = [0, 20, 40, 70, 100, 150, 1_000]
# labels = ["Good", "Slight", "Average", "Heavy", "Severe", "Failure"]

# labels = pd.cut(metadata_df["VB"], bins=bins, labels=labels)
# # print(labels.unique()) # Shows that all labels appear in the dataset
# # labels

In [77]:

bins = [0, 20, 40, 70, 100, 150, 1_000]
labels = ["Good", "Slight", "Average", "Heavy", "Severe", "Failure"]

labels = pd.cut(metadata_df["VB"], bins=bins, labels=labels)
# Remove case 166 and 167 since no data for them

label_dict = {"Good": [], "Slight": [], "Average": [], "Heavy": [], "Severe": [], "Failure": []}

for label in label_dict:
    label_dict[label] = labels[labels==label].index + 1
    

label_dict

{'Good': Index([  4,   6,  18,  19,  20,  21,  22,  24,  33,  34,  35,  36,  37,  46,
         47,  48,  54,  55,  56,  63,  64,  65,  73,  74,  75,  76,  77,  79,
         80,  81,  82,  96,  97,  99, 100, 101, 111, 118, 119, 126, 133, 134,
        135, 147, 148, 156],
       dtype='int64'),
 'Slight': Index([  7,   8,   9,  10,  11,  12,  25,  26,  27,  28,  38,  39,  40,  41,
         42,  43,  49,  50,  51,  57,  58,  66,  67,  83,  84,  86,  87, 102,
        103, 105, 112, 120, 121, 122, 127, 136, 137, 149, 151, 157, 158, 164,
        166],
       dtype='int64'),
 'Average': Index([ 13,  14,  15,  17,  29,  30,  31,  44,  45,  52,  59,  60,  68,  69,
         70,  71,  89,  90,  91,  92,  93, 106, 107, 108, 109, 113, 114, 123,
        129, 130, 138, 139, 140, 152, 159, 160, 161, 167],
       dtype='int64'),
 'Heavy': Index([61, 94, 115, 141, 142, 153], dtype='int64'),
 'Severe': Index([143, 144, 154], dtype='int64'),
 'Failure': Index([145], dtype='int64')}

In [90]:
# from ipywidgets import interact, Dropdown, IntSlider
# import matplotlib.pyplot as plt
# import numpy as np
# from IPython.display import display


# def sTFT(time_seg, filter_index, case_number, signal_location, category):
#     signal_location_dict = {0: "spindle", 1: "table"}
#     loc_str = signal_location_dict[signal_location]
#     case_str = str(case_number).zfill(3)
#     col_name = f"AE_{loc_str}_case{case_str}"

#     case_metadata = metadata_df.loc[case_number-1]
    
#     time = case_metadata["time"]
#     doc = case_metadata["DOC"] # DOC: Depth of Cut

#     print(doc)

#     feed = case_metadata["feed"]	
#     material = case_metadata["material"]
    
#     if col_name not in audio_data.columns:
#         print(f"{col_name} not found in data.")
#         return
    
#     signal = audio_data[col_name].values
#     N = len(signal)
#     Nwindow = N // time_seg
#     time_freq_image = np.zeros((N // 2, time_seg))

#     for time_index in range(time_seg):
#         signal_pad = np.zeros(N)
#         if filter_index < 2:
#             window = filters[filter_index]["function"](Nwindow)
#         else:
#             window = filters[filter_index]["function"](Nwindow, 10)
#         segment = signal[time_index * Nwindow:(time_index + 1) * Nwindow] * window
#         signal_pad[:Nwindow] = segment
#         signal_freq = np.fft.fft(signal_pad)[:N // 2]
#         signal_freq_max = np.max(np.abs(signal_freq))
#         signal_freq = np.abs(signal_freq) / signal_freq_max
#         time_freq_image[:, time_index] = 20 * np.log10(signal_freq)

#     w, h = plt.figaspect(0.4)
#     plt.figure(figsize=(w, h))
#     plt.imshow(time_freq_image[::-1, :], cmap="gray", extent=(0, N / fs * 1000, 0, fs // 2), aspect='auto', clim=(-60, 0))
#     plt.title(f"{category} — Case {case_str} ({loc_str}) {filters[filter_index]['name']} window, window size = {Nwindow}, time = {time}, doc = {doc}, feed = {feed}, material = {material}")
#     plt.xlabel("Time (ms)")
#     plt.ylabel("Frequency (Hz)")
#     plt.show()


# def create_category_plotter(category_name, labels_dict):
#     valid_cases = sorted(labels_dict[category_name])
#     interact(
#         sTFT,
#         time_seg=IntSlider(min=1, max=50, step=1, value=10),
#         filter_index=IntSlider(min=0, max=2, step=1, value=0),
#         case_number=Dropdown(options=valid_cases, description="Case #"),
#         signal_location=IntSlider(min=0, max=1, step=1, value=0),
#         category=widgets.fixed(category_name)
#     )

# # Create one interactive plotter per category
# for category in label_dict.keys():
#     print(f"\n⏬ {category} category visualization")
#     create_category_plotter(category, label_dict)

Important Notes: <br>
* There are 2 different speeds.
* There are 2 different cutting materials.

In [91]:
col_name = "AE_table_case001"
time_seg = 2
filter_index = 2
signal = audio_data[col_name].values
N = len(signal)
Nwindow = N // time_seg
time_freq_image = np.zeros((N // 2, time_seg))

spectrums = []

for time_index in range(time_seg):
    signal_pad = np.zeros(N)
    if filter_index < 2:
        window = filters[filter_index]["function"](Nwindow)
    else:
        window = filters[filter_index]["function"](Nwindow, 10)
    segment = signal[time_index * Nwindow:(time_index + 1) * Nwindow] * window
    signal_pad[:Nwindow] = segment
    signal_freq = np.fft.fft(signal_pad)[:N // 2]
    signal_freq_max = np.max(np.abs(signal_freq))
    signal_freq = np.abs(signal_freq) / signal_freq_max
    time_freq_image[:, time_index] = 20 * np.log10(signal_freq)

    print(time_freq_image)

[[  0.           0.        ]
 [ -0.83860648   0.        ]
 [ -3.38120735   0.        ]
 ...
 [-61.98040542   0.        ]
 [-61.66986878   0.        ]
 [-61.12425159   0.        ]]
[[  0.           0.        ]
 [ -0.83860648  -0.90164208]
 [ -3.38120735  -3.65451356]
 ...
 [-61.98040542 -65.66571249]
 [-61.66986878 -69.86007702]
 [-61.12425159 -73.54143617]]


In [92]:
def sTFT(time_seg, filter_index, case_number, sensor_type, category):
    case_str = str(case_number).zfill(3)
    col_name = f"{sensor_type}_case{case_str}"

    case_metadata = metadata_df.loc[case_number - 1]
    time = case_metadata["time"]
    doc = case_metadata["DOC"]
    feed = case_metadata["feed"]
    material = case_metadata["material"]

    if col_name not in all_data.columns:
        print(f"{col_name} not found in data.")
        return

    signal = all_data[col_name].values
    N = len(signal)
    Nwindow = N // time_seg
    time_freq_image = np.zeros((N // 2, time_seg))

    for time_index in range(time_seg):
        signal_pad = np.zeros(N)
        if filter_index < 2:
            window = filters[filter_index]["function"](Nwindow)
        else:
            window = filters[filter_index]["function"](Nwindow, 10)
        segment = signal[time_index * Nwindow:(time_index + 1) * Nwindow] * window
        signal_pad[:Nwindow] = segment
        signal_freq = np.fft.fft(signal_pad)[:N // 2]
        signal_freq_max = np.max(np.abs(signal_freq))
        signal_freq = np.abs(signal_freq) / signal_freq_max
        time_freq_image[:, time_index] = 20 * np.log10(signal_freq)

    w, h = plt.figaspect(0.4)
    plt.figure(figsize=(w, h))
    plt.imshow(time_freq_image[::-1, :], cmap="gray", extent=(0, N / fs * 1000, 0, fs // 2), aspect='auto', clim=(-60, 0))
    plt.title(f"{category} — Case {case_str} ({sensor_type}) {filters[filter_index]['name']} window, window size = {Nwindow}, time = {time}, doc = {doc}, feed = {feed}, material = {material}")
    plt.xlabel("Time (ms)")
    plt.ylabel("Frequency (Hz)")
    plt.show()

In [93]:
def create_category_plotter(category_name, labels_dict):
    valid_cases = sorted(labels_dict[category_name])
    sensor_options = ["AE_spindle", "AE_table", "vib_spindle", "vib_table", "smcAC", "smcDC"]
    
    interact(
        sTFT,
        time_seg=IntSlider(min=1, max=50, step=1, value=10),
        filter_index=IntSlider(min=0, max=2, step=1, value=0),
        case_number=Dropdown(options=valid_cases, description="Case #"),
        sensor_type=Dropdown(options=sensor_options, description="Sensor"),
        category=widgets.fixed(category_name)
    )

# Create one interactive plotter per category
for category in label_dict.keys():
    print(f"\n⏬ {category} category visualization")
    create_category_plotter(category, label_dict)


⏬ Good category visualization


interactive(children=(IntSlider(value=10, description='time_seg', max=50, min=1), IntSlider(value=0, descripti…


⏬ Slight category visualization


interactive(children=(IntSlider(value=10, description='time_seg', max=50, min=1), IntSlider(value=0, descripti…


⏬ Average category visualization


...

interactive(children=(IntSlider(value=10, description='time_seg', max=50, min=1), IntSlider(value=0, descripti…


⏬ Severe category visualization


interactive(children=(IntSlider(value=10, description='time_seg', max=50, min=1), IntSlider(value=0, descripti…


⏬ Failure category visualization


interactive(children=(IntSlider(value=10, description='time_seg', max=50, min=1), IntSlider(value=0, descripti…